Pre-processing
* Visualization
1. Unix time
2. Encoding of categorical features
3. Temporal ordering
4. Aditional features:
- Previous event
- Next event
- Day of the week
- Time of day
- Event duration
5. Separate 80-20 
- Visualization
6. Get rid of overlap

In [44]:
import os
import psutil
import numpy as np  # import auxiliary library, typical idiom
import pandas as pd  # import the Pandas library, typical idiom
from pandas import read_csv
import statsmodels.api as sm
import time
import pm4py
from datetime import datetime
from datetime import date
from datetime import datetime
from datetime import timedelta

from numba import jit

from sklearn.linear_model import LinearRegression  # for linear regression
from sklearn import linear_model
from sklearn.cluster import KMeans  # for clustering
from sklearn.tree import DecisionTreeClassifier  # for decision tree mining
from sklearn.metrics import mean_absolute_error, confusion_matrix, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf 
import statsmodels.api as sm
from statsmodels.graphics.gofplots import ProbPlot
from matplotlib import pyplot
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

In [29]:
file_export = 'export2018.csv'
data = pd.read_csv(file_export)

In [30]:
data = data.sort_values(by=['case','startTime'])

In [34]:
#Duration
@jit(parallel = True)
def calculator_nb(case, startTime):
    res = np.empty(len(case), dtype=object)
    idx = 0
    for _ in case:
        if (idx+1 >= len(case)):
            break

        if (case[idx + 1] == case[idx]):
            res[idx] = startTime[idx + 1]
        else:
            res[idx] = startTime[idx]

        idx+=1
    return res

data['completeTime'] = calculator_nb(data['case'].values, data['startTime'].values)
data.at[317373, 'completeTime'] = data.at[317373, 'startTime']

data['startTime'] =  pd.to_datetime(data['startTime'])
data['completeTime'] =  pd.to_datetime(data['completeTime'])
data['duration'] = data['completeTime'] - data['startTime']
#to turn duration into seconds:
duration = data['duration']
duration = duration / np.timedelta64(1, 's')
data['duration'] = duration

C:\Users\mikok\AppData\Local\Temp\ipykernel_18572\696655479.py:2: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_18572\696655479.py", line 4:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Users\mikok\AppData\Local\Temp\ipykernel_18572\696655479.py:2: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_18572\696655479.py", line 4:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Users\mikok\AppData\Local\Programs\Python\Python310\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "calculator_nb" w

In [37]:
#Next event
@jit(parallel = True)
def calculator_nb(case, event):
    res = np.empty(len(case), dtype=object)
    idx = 0
    for _ in case:
        if (idx+1 >= len(case)):
            break
       
        if (case[idx + 1] == case[idx]):
            res[idx] = event[idx + 1]

        idx+=1
    return res

data['next_event'] = calculator_nb(data['case'].values, data['event'].values)

C:\Users\mikok\AppData\Local\Temp\ipykernel_18572\1360192189.py:2: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_18572\1360192189.py", line 4:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Users\mikok\AppData\Local\Temp\ipykernel_18572\1360192189.py:2: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_18572\1360192189.py", line 4:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Users\mikok\AppData\Local\Programs\Python\Python310\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "calculator_n

In [38]:
#Previous event
@jit(parallel = True)
def calculator_nb(case, event):
    res = np.empty(len(case), dtype=object)
    idx = 0
    for _ in case:
        if (idx+1 >= len(case)):
            break
       
        if (case[idx + 1] == case[idx]):
            res[idx + 1] = event[idx]

        idx+=1
    return res

data['prev_event'] = calculator_nb(data['case'].values, data['event'].values)

C:\Users\mikok\AppData\Local\Temp\ipykernel_18572\1972138414.py:2: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_18572\1972138414.py", line 4:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Users\mikok\AppData\Local\Temp\ipykernel_18572\1972138414.py:2: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_18572\1972138414.py", line 4:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Users\mikok\AppData\Local\Programs\Python\Python310\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "calculator_n

In [39]:
#Removing null values
data['next_event'] = data['next_event'].fillna(value='None')
data['prev_event'] = data['prev_event'].fillna(value='None')

In [40]:
#unix time
pd.set_option('display.float_format', lambda x: '%.3f' % x)

data['startTime'] = pd.to_datetime(data['startTime'], dayfirst=True)
unixTransform = lambda x: time.mktime(x.timetuple())
data["UNIX_starttime"] = data["startTime"].apply(unixTransform).astype(int)

data['completeTime'] = pd.to_datetime(data['completeTime'], dayfirst=True)
unixTransform = lambda x: time.mktime(x.timetuple())
data["UNIX_completeTime"] = data["completeTime"].apply(unixTransform).astype(int)

#data['REG_DATE'] = pd.to_datetime(data['REG_DATE'], dayfirst=True)
#unixTransform = lambda x: time.mktime(x.timetuple())
#data["UNIX_REG_DATE"] = data["REG_DATE"].apply(unixTransform).astype(int)

#print(data)

In [42]:
#Day of the week
data['weekday'] = data['startTime'].dt.dayofweek

In [45]:
#encoding of categorical data
ordinal_encoder = OrdinalEncoder()
label_encoder = LabelEncoder()
data['enc_event'] = ordinal_encoder.fit_transform(data[['event']]).astype(int)

In [43]:
#ensure we have acces to orignal indexing to keep track of the order of events in a process
data['original index'] = data.index

#sorting on time
data.sort_values(by = "UNIX_starttime", ignore_index=True)

,case,event,startTime,completeTime,penalty_JLP1,penalty_JLP3,number_parcels,penalty_JLP2,penalty_JLP5,year,...,docid_uuid,org:resource,success,duration,next_event,prev_event,UNIX_starttime,UNIX_completeTime,weekday,original index
0,980452d16c80c2c3,mail valid,2014-05-04 00:00:00.000,2015-04-24 00:00:00.000,False,False,32,False,False,2015,...,93F2F9CA-6F07-4E87-8107-4852A7B1CBA2,0;n/a,True,30672000.000,mail income,mail valid,1399154400,1429826400,6,63730
1,980452d16c80c2c3,mail valid,2014-05-04 00:00:00.000,2014-05-04 00:00:00.000,False,False,32,False,False,2015,...,93F2F9CA-6F07-4E87-8107-4852A7B1CBA2,0;n/a,True,0.000,mail valid,mail valid,1399154400,1399154400,6,63729
2,cb3425ce193199d7,mail valid,2014-05-04 00:00:00.000,2014-05-04 00:00:00.000,False,False,5,False,False,2015,...,990E93F0-3BDD-464C-8102-F201E9F5EEE4,0;n/a,True,0.000,mail valid,None,1399154400,1399154400,6,594649
3,980452d16c80c2c3,mail valid,2014-05-04 00:00:00.000,2014-05-04 00:00:00.000,False,False,32,False,False,2015,...,2118A637-DCB0-4382-BC47-93BF0CF535AC,0;n/a,True,0.000,mail valid,None,1399154400,1399154400,6,63727
4,cb3425ce193199d7,mail valid,2014-05-04 00:00:00.000,2014-05-04 00:00:00.000,False,False,5,False,False,2015,...,F94BE88D-511C-44B8-B2CE-988A147E7DF8,0;n/a,True,0.000,mail valid,mail valid,1399154400,1399154400,6,594651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2514261,8cd07a136f401349,calculate,2018-01-19 12:27:24.824,2018-01-19 12:27:24.824,False,False,37,False,False,2016,...,D555F997-BDE0-471A-99AC-5CC0E1BD650F,75992a,True,0.000,None,begin editing,1516361244,1516361244,4,1020230
2514262,dc56a531f5aeb3b0,save,2018-01-19 12:51:34.000,2018-01-19 12:51:34.000,False,False,49,False,False,2016,...,24CD9E6F-6A0D-4C41-8D76-D3BE35B82681,8d8538,True,0.000,None,finish payment,1516362694,1516362694,4,1553199
2514263,7b405bc5144c14c9,save,2018-01-19 12:57:05.000,2018-01-19 13:02:32.000,False,False,43,False,False,2016,...,6EF39194-5730-4F6E-966C-C6B1D7A7DBC6,8d8538,True,327.000,save,finish payment,1516363025,1516363352,4,1189623
2514264,7b405bc5144c14c9,save,2018-01-19 13:02:32.000,2018-01-19 13:02:32.000,False,False,43,False,False,2016,...,6EF39194-5730-4F6E-966C-C6B1D7A7DBC6,8d8538,True,0.000,None,save,1516363352,1516363352,4,1189624


In [46]:
#separation
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, shuffle=False)

In [47]:
#removing overlap - if case is in both datasets, remove

train_cases = train['case'].unique().tolist()
test_cases = test['case'].unique().tolist()

intersect_list = list(set(train_cases).intersection(test_cases))

In [48]:
#only removes first value in intersect list (needs modification for multiple overlaping values)

#train = train[train['case'] != intersect_list[0]]
#test = test[test['case'] != intersect_list[0]]

#works for more values

train = train[train['case'].isin(intersect_list) == False]
test = test[test['case'].isin(intersect_list) == False]

In [ ]:
#visualization (no need to include in main code, but might be useful)
log = pm4py.read_xes('bpi2018.xes')

process_tree = pm4py.discover_tree_inductive(log)
pm4py.view_process_tree(process_tree)

In [ ]:
#separation visualisation

g = sns.scatterplot(x="UNIX_starttime", y="case", hue="enc_event", data=data, palette='colorblind', legend=False)

#add lines for separation - horizontal and vertical
